## How reliable are SAEs?
Inspired by code from

https://github.com/JoshEngels/SAE-Probes/

https://github.com/JoshEngels/SAE-Dark-Matter/

https://github.com/ejmichaud/feature-geometry/brain

In [1]:
%pip install transformer_lens

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
  

In [1]:
from transformer_lens import HookedTransformer

In [3]:
%pip install sae_lens transformers datasets torch sae_bench scikit-learn natsort

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.1/920.1 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.8/291.8 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 102.8 MB/s eta 0:00:0

In [2]:
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, ActivationCache, HookedTransformerConfig, patching, FactoredMatrix
from transformer_lens.components import Embed, Unembed, LayerNorm, MLP

In [4]:
import yaml
import glob
import pandas as pd
import torch
from torch import Tensor
import einops
from tqdm.notebook import tqdm
import webbrowser




torch.set_grad_enabled(False)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_name = "gemma-2-2b"

MAIN = __name__ == "__main__"


#### Gemma2(2B) details
d_model = 2304
<br>
layers 26 <br>
uses GeGLU <br>
Feedforward dim = 18432<br>
Head type = GQA <br>
Heads = 8<br>
KV heads 4<br>
head dim = 256<br>
global attn 8192<br>
sliding window 4096<br>
vocab size 256128

In [ ]:
#list of all activation paths in SAE-Dark-Matter/gemma_sae_pickles/pretrained.yaml
#https://arxiv.org/abs/2408.05147 --- gemma 2b SAE = gemma scope

experiment_details=
"""
gemma-2b-res-jb:
  repo_id: jbloom/Gemma-2b-Residual-Stream-SAEs
  model: gemma-2b
  conversion_func: null
  links:
    model: https://huggingface.co/google/gemma-2b
    dashboards: https://www.neuronpedia.org/gemma2b-res-jb
  saes:
  - id: blocks.0.hook_resid_post
    path: gemma_2b_blocks.0.hook_resid_post_16384_anthropic
    neuronpedia: gemma-2b/0-res-jb
    variance_explained: 0.999
    l0: 47.0
  - id: blocks.6.hook_resid_post
    path: gemma_2b_blocks.6.hook_resid_post_16384_anthropic_fast_lr
    neuronpedia: gemma-2b/6-res-jb
    variance_explained: 0.71
    l0: 56.0

gemma-scope-2b-pt-res:
  repo_id: google/gemma-scope-2b-pt-res
  model: gemma-2-2b
  conversion_func: gemma_2
  saes:
  - id: layer_0/width_16k/average_l0_105
    path: layer_0/width_16k/average_l0_105
    l0: 105
  - id: layer_5/width_16k/average_l0_143
    path: layer_5/width_16k/average_l0_143
    l0: 143
"""

In [6]:
size = "2b"
extension = "res"
layersCat0 = range(0,5)
layersCat1 = range(10,15)
layersCat2 = range(21,26)
sae_widthSweepthrus = [int(2**14),int(2**15),int(2**17),int(2**18),int(2**19)]
average_lzero = 50 #lzero needs to be similar for comparision across layers
#read the available sparsity from the yml file
lcat0xwidth, lcat1xwidth, lcat2xwidth = {}, {}, {}

f = open("/content/pretrained.yml","r")
pretrained = yaml.load(f, Loader=yaml.FullLoader)
for sae in pretrained[f"gemma-scope-{size}-pt-{extension}"]["saes"]:
  layer,width,l0 = sae["id"].split("/")
  layer = int(layer.split("_")[1])
  width = width.split("_")[1]
  l0 = int(l0.split("_")[2])
  width = {"16k": int(2**14), "32k": int(2**15), "65k": int(2**16), "131k": int(2**17), "262k": int(2**18), "524k": int(2**19), "1m": int(2**20)}[width.split("_")[-1]]
  if (layer in layersCat0) and (width in sae_widthSweepthrus):
    lcat0xwidth.setdefault((layer,width),[]).append(l0)
  if (layer in layersCat1) and (width in sae_widthSweepthrus):
    lcat1xwidth.setdefault((layer,width),[]).append(l0)
  if (layer in layersCat2) and (width in sae_widthSweepthrus):
    lcat2xwidth.setdefault((layer,width),[]).append(l0)

lcat0xwidth

{(0, 16384): [105, 13, 226, 25, 46],
 (1, 16384): [10, 102, 20, 250, 40],
 (2, 16384): [13, 141, 142, 24, 304, 53],
 (3, 16384): [14, 142, 28, 315, 59],
 (4, 16384): [124, 125, 17, 281, 31, 60]}

In [8]:
lcat1xwidth

{(10, 16384): [166, 21, 39, 395, 77],
 (11, 16384): [168, 22, 393, 41, 80],
 (12, 16384): [176, 22, 41, 445, 82],
 (13, 16384): [173, 23, 403, 43, 83, 84],
 (14, 16384): [173, 23, 388, 43, 83, 84],
 (12, 262144): [11, 121, 21, 243, 36, 67],
 (12, 32768): [12, 155, 22, 360, 40, 76],
 (12, 524288): [115, 22, 227, 29, 46, 65]}

In [9]:
lcat2xwidth

{(21, 16384): [139, 22, 301, 38, 70],
 (22, 16384): [147, 21, 349, 38, 72],
 (23, 16384): [157, 21, 38, 404, 74, 75],
 (24, 16384): [158, 20, 38, 457, 73],
 (25, 16384): [116, 16, 28, 285, 55]}

In [10]:
size = "2b"
extension = "res"
#setup the layers required for expt
layersCat0 = range(0,5)
layersCat1 = range(10,15)
layersCat2 = range(21,26)
sae_widthSweepthrus = [int(2**14),int(2**15),int(2**17),int(2**18),int(2**19)]
average_lzero = 50 #100 lzero needs to be similar for comparision across layers
#read the available sparsity from the yml file
lcat0xwidth, lcat1xwidth, lcat2xwidth = {}, {}, {}

f = open("/content/pretrained.yml","r")
pretrained = yaml.load(f, Loader=yaml.FullLoader)
for sae in pretrained[f"gemma-scope-{size}-pt-{extension}"]["saes"]:
  layer,width,l0 = sae["id"].split("/")
  layer = int(layer.split("_")[1])
  width = width.split("_")[1]
  l0 = int(l0.split("_")[2])
  width = {"16k": int(2**14), "32k": int(2**15), "65k": int(2**16), "131k": int(2**17), "262k": int(2**18), "524k": int(2**19), "1m": int(2**20)}[width.split("_")[-1]]
  if (layer in layersCat0) and (width in sae_widthSweepthrus):
    lcat0xwidth.setdefault((layer,width),[]).append(l0)
  if (layer in layersCat1) and (width in sae_widthSweepthrus):
    lcat1xwidth.setdefault((layer,width),[]).append(l0)
  if (layer in layersCat2) and (width in sae_widthSweepthrus):
    lcat2xwidth.setdefault((layer,width),[]).append(l0)


In [ ]:
#input for experiments
#saeprobe
dataset_names = glob.glob("data/cleaned_data/*.csv")


#saeDarkMatter
dataset_name = "monology/pile-uncopyrighted"
save_dir_base = f"expt1matrices/gemma_{size}_sae_scaling"
dataset = load_dataset(dataset_name, split=split, streaming=streaming)
dataset = dataset.shuffle(seed=42, buffer_size=10000)
data = [x["text"] for x in iter(dataset)]


def tokenized_batch():
  batch = []
  while len(batch) < batch_size:
    next_text = next(data)
    tokenized = model.tokenizer(
            next_text,
            return_tensors="pt",
            max_length=ctx_len,
            padding=False,
            truncation=True,
        )
    if tokenized["input_ids"].shape[1] == ctx_len:
      batch.append(tokenized)
    return torch.cat([x["input_ids"] for x in batch], dim=0)


In [ ]:
#model activation
model = HookedTransformer.from_pretrained("google/gemma-2-2b", device=device)
tokenizer = model.tokenizer
tokenizer.truncation_side='left'
tokenizer.padding_side='right'

hook_names = [f"blocks.{layer}.hook_resid_post" for layer in layersCat0] #layersCat1, layersCat2
context_len = 1024
batch_size = 10
num_contexts = 300

def get_activations_and_loss(input_tokens):
    losses, cache = model.run_with_cache(
        input_tokens, names_filter=lambda name: name in hook_names, return_type="loss", loss_per_token=True
    )
    acts = [cache[hook_name] for hook_name in hook_names]
    return losses, acts

#model act \in batch*num_interlayers x dimensions
cat0acts, cat1acts, cat2acts = [], [], []
label0, label1, label2 = [],[],[]
cat0loss, cat1loss, cat2loss = [], [], []
all_tokens = []

bar = tqdm(range(0, num_contexts, batch_size))
for i in bar:

    input_tokens = tokenized_batch().to(device)
    all_tokens.append(input_tokens.to("cpu"))
    losses, acts = get_activations_and_loss(input_tokens)

    for index, (layer, layer_acts) in enumerate(zip(layers, acts)):
        layer_act_norms = layer_acts.norm(dim=-1)
        if layer in layersCat0:
          cat0acts[index].append(layer_acts.to("cpu"))
          cat0loss[index].append(layer_acts.to("cpu"))
          label0[index].append(layer)
        if layer in layersCat1:
          cat1acts[index].append(layer_acts.to("cpu"))
          cat1loss[index].append(layer_acts.to("cpu"))
          label1[index].append(layer)
        if layer in layersCat2:
          cat2acts[index].append(layer_acts.to("cpu"))
          cat2loss[index].append(layer_acts.to("cpu"))
          label2[index].append(layer)

def save_so_far():
  all_tokens_cat = torch.cat(all_tokens, dim=0)
  torch.save(all_tokens_cat, os.path.join(save_dir_base, "cat0tokens.pt"))
  all_tokens_cat.numpy().tofile(os.path.join(save_dir_base, f"cat0tokens.npy"))
  label0.tofile(os.path.join(save_dir_base, f"cat0labels.npy"))
  label1.tofile(os.path.join(save_dir_base, f"cat1labels.npy"))
  label2.tofile(os.path.join(save_dir_base, f"cat2labels.npy"))


  for layer, layer_acts in zip(layersCat0,cat0acts):
    layer_acts_cat = torch.cat(layer_acts, dim=0)
    layer_acts_cat.numpy().tofile(os.path.join(save_dir_base, f"acts{layer}layerCat0.npy"))
  for layer, layer_acts in zip(layersCat1,cat1acts):
    layer_acts_cat = torch.cat(layer_acts, dim=0)
    layer_acts_cat.numpy().tofile(os.path.join(save_dir_base, f"acts{layer}layerCat1.npy"))
  for layer, layer_acts in zip(layersCat2,cat2acts):
    layer_acts_cat = torch.cat(layer_acts, dim=0)
    layer_acts_cat.numpy().tofile(os.path.join(save_dir_base, f"acts{layer}layerCat2.npy"))


In [1]:
#sae activations
from sae_lens import SAE

def getandsave_sae_acts(sae_name,acts, size, layer_type):
  res, all_l0s, all_loss = [],[],[]
  sae, cfg_dict, sparsity = SAE.from_pretrained(
    release=f"gemma-scope-{size}-pt-{layer_type}",
    sae_id=sae_name,
    device=device,
    )
  hook_name = sae.cfg.hook_name
  print(hook_name)
  num_contexts, ctx_len, dim = acts.shape
  total_tokens = num_contexts * ctx_len
  print(f"Total tokens: {total_tokens / 1e6:.2f}M")
  bar = tqdm(range(0, num_contexts, batch_size))
  for i in bar:
      acts_batch_cpu = acts[i : i + batch_size]
      acts_batch = acts_batch_cpu.clone().to(device)

      feature_acts = sae.encode(acts_batch)
      reconstructions = sae.decode(feature_acts)
      feature_acts_cpu = feature_acts.to("cpu")
      print(f"sae features shape{feature_acts_cpu.shape}")
      actual_lzees = (feature_acts_cpu > 0).sum(dim=-1)
      feature_acts_norms = feature_acts_cpu.norm(dim=-1)

      for j in range(batch_size):
          feature_acts_j = feature_acts_cpu[j]
          nonzero_feature_indices_j = torch.nonzero(feature_acts_j, as_tuple=True)
          nonzero_feature_values_j = feature_acts_j[nonzero_feature_indices_j]
          res = [nonzero_feature_indices_j[0], nonzero_feature_indices_j[1], nonzero_feature_values_j]

      # Get reconstruction error
      reconstructions_cpu = reconstructions.to("cpu")
      reconstruction_errors = reconstructions_cpu - acts_batch_cpu


      all_l0s.append(actual_lzees.to("cpu"))
      all_loss.append(reconstruction_errors.to("cpu"))
      all_feature_act_norms.append(feature_acts_norms.to("cpu"))
      bar.set_description(
          f"Num tokens: {len(all_loss) * batch_size * ctx_len / 1e6:.2f}M"
      )
      return res, res_norm, all_l0s, all_loss

#sae feature reps \in batch*num_interlayers x dimensions
saeActsCat0, saeActsCat1, saeActsCat2  = [], [], []
label_sae0, label_sae1, label_sae2 = [],[],[]
sae0loss, sae1loss, sae2loss = [], [], []


for layer in layerCat0:
  acts_shape = [300, 1024, 2304] #supposed to be num_contexts, ctx_len, dim
  label_sae0.append(layer)
  acts = torch.from_file(f"{save_dir_base}/acts{layer}layerCat0.npy", shared=False, size=np.prod(acts_shape)).view(*acts_shape)
  for curr_width in [int(2**14)]: #sae_widthSweepthrus = int(2**14);hardcoded for the time being
    lzee = min(lcat0xwidth[(layer,curr_width)], key = lambda x: abs(x-average_lzero))
    print(f"sparsity = {lzee}")
    sae_name = f"layer_{layer}/width_{curr_width}/average_l0_{lzee}"
    save_dir = f"{save_dir_base}/{sae_name}"
    os.makedirs(save_dir, exist_ok=True)
    saeActsCat0, saeNormsCat0, sae0l0s, sae0loss = getandsave_sae_acts(sae_name, acts, "2b", "res")
    torch.save(sae0l0s, f"{save_dir}/sae0_l0slayer{layer}.pt")
    sae0loss_cat = torch.cat(sae0loss, dim=0)
    sae0loss_cat.numpy().tofile(f"{save_dir}/sae0error_vecs_layer{layer}.npy")
    sae0actsNormcat = torch.cat(saeNormsCat0, dim=0)
    torch.save(
          sae0actsNormcat, f"{save_dir}/actfeatnormSae0_layer{layer}.pt"
    )
    torch.save(saeActsCat0, f"{save_dir}/actfeatSae0_layer{layer}.pt")

for layer in layerCat1:
  acts_shape = [300, 1024, 2304] #supposed to be num_contexts, ctx_len, dim
  label_sae1.append(layer)
  acts = torch.from_file(f"{save_dir_base}/acts{layer}layerCat1.npy", shared=False, size=np.prod(acts_shape)).view(*acts_shape)
  for curr_width in [int(2**14)]: #sae_widthSweepthrus = int(2**14);hardcoded for the time being
    lzee = min(lcat1xwidth[(layer,curr_width)], key = lambda x: abs(x-average_lzero))
    print(f"sparsity = {lzee}")
    sae_name = f"layer_{layer}/width_{curr_width}/average_l0_{lzee}"
    save_dir = f"{save_dir_base}/{sae_name}"
    os.makedirs(save_dir, exist_ok=True)
    saeActsCat1, saeNormsCat1, sae1l0s, sae1loss = getandsave_sae_acts(sae_name, acts, "2b", "res")
    torch.save(sae1l0s, f"{save_dir}/sae1_l0slayer{layer}.pt")
    sae1loss_cat = torch.cat(sae1loss, dim=0)
    sae1loss_cat.numpy().tofile(f"{save_dir}/sae1error_vecs_layer{layer}.npy")
    sae1actsNormcat = torch.cat(saeNormsCat1, dim=0)
    torch.save(
          sae1actsNormcat, f"{save_dir}/actfeatnormSae1_layer{layer}.pt"
    )
    torch.save(saeActsCat1, f"{save_dir}/actfeatSae1_layer{layer}.pt")

for layer in layerCat2:
  acts_shape = [300, 1024, 2304] #supposed to be num_contexts, ctx_len, dim
  label_sae2.append(layer)
  acts = torch.from_file(f"{save_dir_base}/acts{layer}layerCat2.npy", shared=False, size=np.prod(acts_shape)).view(*acts_shape)
  for curr_width in [int(2**14)]: #sae_widthSweepthrus = int(2**14);hardcoded for the time being
    lzee = min(lcat2xwidth[(layer,curr_width)], key = lambda x: abs(x-average_lzero))
    print(f"sparsity = {lzee}")
    sae_name = f"layer_{layer}/width_{curr_width}/average_l0_{lzee}"
    save_dir = f"{save_dir_base}/{sae_name}"
    os.makedirs(save_dir, exist_ok=True)
    saeActsCat2, saeNormsCat2, sae2l0s, sae2loss = getandsave_sae_acts(sae_name, acts, "2b", "res")
    torch.save(sae2l0s, f"{save_dir}/sae2_l0slayer{layer}.pt")
    sae2loss_cat = torch.cat(sae2loss, dim=0)
    sae2loss_cat.numpy().tofile(f"{save_dir}/sae2error_vecs_layer{layer}.npy")
    sae2actsNormcat = torch.cat(saeNormsCat2, dim=0)
    torch.save(
          sae2actsNormcat, f"{save_dir}/actfeatnormSae2_layer{layer}.pt"
    )
    torch.save(saeActsCat2, f"{save_dir}/actfeatSae2_layer{layer}.pt")



ModuleNotFoundError: No module named 'sae_lens'